In [79]:
import numpy as np
from astropy.stats import mad_std
from photutils.detection import DAOStarFinder
from scipy.optimize import curve_fit

def fwhm(image, fwhm=3.0, threshold_multiplier=5.0, size=10):
    image_data = image
    background_noise = mad_std(image_data[1])
    first_layer = image_data.reshape(512, 512)
    daofind = DAOStarFinder(fwhm=fwhm, threshold=threshold_multiplier * background_noise)
    sources = daofind(first_layer)

    if sources is None:
        print("Yıldızlar bulunamadı....:(")
        print(1)
        return None
   
    
    try:
        sorted_sources = sorted(sources, key=lambda x: x['flux'], reverse=True)
        top_3_stars = sorted_sources[:3]
        if(len(top_3_stars) < 3):
            print(top_3_stars)
            return None
    except:
        print(3)
        return None
    
    def gaussian_1d(x, amp, mean, stddev):
        return amp * np.exp(-0.5 * ((x - mean) / stddev) ** 2)
    
    fwhm_values = []
    for star in top_3_stars: 
        x, y = int(star['xcentroid']), int(star['ycentroid'])
        cutout = first_layer[y - size:y + size, x - size:x + size]

        try:
            profile = cutout[size, :]
        except:
            return None
            continue
        x_vals = np.arange(len(profile))
        
        if(profile.shape[0] < 20):
            print(profile.shape)
            print(5)
            return None
            continue
        
        try:
            popt, _ = curve_fit(gaussian_1d, x_vals, profile, p0=[np.max(profile), size, 1.0])
            amp, mean, stddev = popt
            fwhm = 2 * np.sqrt(2 * np.log(2)) * stddev
            fwhm_values.append(fwhm)
        except RuntimeError: 
            print(6)
            return None

    valid_fwhm_values = [fwhm for fwhm in fwhm_values if not np.isnan(fwhm)]
    total_fwhm = sum(valid_fwhm_values)
    num_valid_stars = len(valid_fwhm_values)
    average_fwhm = total_fwhm / num_valid_stars if num_valid_stars > 0 else np.nan

    return valid_fwhm_values